##### Домашнее задание

## Урок №10 - Библиотека Request и REST API

#### Работу выполнил: Зензеров Никита Евгеньевич

Вам будут даны файлы с событиями:

+ Установки:
    + ID пользователя,
    + IP-адрес установки.
+ Платежа:
    + ID пользователя,
    + сумма платежа.
    
Нужно будет посчитать средний чек пользователя из разных стран (средний чек — ARPU).

Бонусное (на оценку не влияет): попробуйте распределить страны по 3-м категориям (на
tier-1, tier-2, tier-3), на основании размера среднего чека от пользователя из их страны.

In [1]:
# Импортируем библиотеки

import pandas as pd

import numpy as np

import requests

import json

import time

In [2]:
# Подгружаем данные и выводим на экран

installs = pd.read_csv('les10_installs.csv')

pay = pd.read_csv('les10_payments.csv')

display(installs)
display(pay)

,event_time,event_date,user_id,ip,device_type
0,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X
1,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11
2,2021-01-01 03:33:58,2021-01-01,216152,74.56.1.222,iPad 5
3,2021-01-01 03:47:43,2021-01-01,216153,70.93.185.219,IPod touch 7
4,2021-01-01 03:58:57,2021-01-01,214545,174.62.104.183,iPhone XS
...,...,...,...,...,...
995,2021-01-05 17:49:17,2021-01-05,131706,176.59.50.238,xiaomi-Redmi Note 7
996,2021-01-05 17:51:07,2021-01-05,216756,46.211.46.213,Xiaomi-Redmi 5A
997,2021-01-05 17:53:50,2021-01-05,106302,88.155.87.158,Xiaomi-Redmi 7A
998,2021-01-05 18:03:26,2021-01-05,118279,46.211.46.213,Xiaomi-Redmi 5A


,event_time,event_date,user_id,revenue_usd
0,2021-01-01 03:24:11,2021-01-01,216151,9.000000
1,2021-01-01 03:24:49,2021-01-01,216151,10.000000
2,2021-01-01 03:46:11,2021-01-01,216151,6.000000
3,2021-01-01 03:46:56,2021-01-01,216151,8.000000
4,2021-01-01 04:49:01,2021-01-01,216151,7.000000
...,...,...,...,...
293,2021-03-29 19:36:02,2021-03-29,155900,8.990000
294,2021-04-06 16:59:05,2021-04-06,215753,1.285853
295,2021-04-16 10:07:19,2021-04-16,185299,5.255567
296,2021-04-16 21:50:03,2021-04-16,215753,5.268238


In [3]:
# Проверяем тип данных

installs.dtypes

event_time     object
event_date     object
user_id         int64
ip             object
device_type    object
dtype: object

In [4]:
pay.dtypes

event_time      object
event_date      object
user_id          int64
revenue_usd    float64
dtype: object

In [5]:
# Проверим на дубликаты

installs.duplicated().sum()

0

In [6]:
pay.duplicated().sum()

0

In [7]:
# Получим общую таблицу с датой установки, платежа, ip и доходом

pays = installs.merge(
    pay,
    on='user_id',
    how='inner',
    suffixes=('_installs', '_payments')
)
pays

,event_time_installs,event_date_installs,user_id,ip,device_type,event_time_payments,event_date_payments,revenue_usd
0,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,2021-01-05 09:52:02,2021-01-05,10.000000
1,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,2021-01-07 03:49:32,2021-01-07,8.000000
2,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,2021-01-08 07:18:57,2021-01-08,8.000000
3,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11,2021-01-01 03:24:11,2021-01-01,9.000000
4,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11,2021-01-01 03:24:49,2021-01-01,10.000000
...,...,...,...,...,...,...,...,...
562,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-16 16:23:17,2021-01-16,1.346895
563,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-16 21:48:23,2021-01-16,1.346895
564,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-18 14:23:39,2021-01-18,0.660628
565,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-18 23:16:05,2021-01-18,5.391229


In [8]:
# Получим каждый уникальный ip, с которого проводился платёж.

pays_group = pays.groupby(['user_id', 'ip'], as_index=False).agg({'revenue_usd':'sum'})
pays_group

,user_id,ip,revenue_usd
0,101559,46.211.82.121,38.496611
1,101559,88.155.87.158,38.496611
2,101559,94.153.28.71,38.496611
3,101559,94.153.39.188,76.993222
4,101559,94.153.51.41,38.496611
...,...,...,...
87,216808,107.77.197.172,248.000000
88,216817,174.222.6.202,5.000000
89,216820,76.169.223.121,43.000000
90,216821,46.72.217.127,3.339133


In [9]:
# Загоним ip в список для дальнейшей работы

ip_list = pays_group['ip'].tolist()
ip_list

['46.211.82.121',
 '88.155.87.158',
 '94.153.28.71',
 '94.153.39.188',
 '94.153.51.41',
 '194.44.191.146',
 '46.211.52.51',
 '88.155.87.158',
 '198.45.117.113',
 '37.151.80.79',
 '95.82.113.175',
 '95.82.113.175',
 '79.170.137.163',
 '79.170.138.60',
 '80.250.77.52',
 '95.129.169.201',
 '95.129.169.212',
 '90.188.177.197',
 '176.212.96.61',
 '176.59.53.125',
 '67.6.130.230',
 '176.59.55.106',
 '176.98.70.86',
 '67.44.208.167',
 '208.98.136.204',
 '188.234.197.40',
 '74.76.231.192',
 '178.176.74.170',
 '172.58.29.110',
 '46.148.132.39',
 '93.157.175.97',
 '109.252.57.7',
 '46.242.15.57',
 '94.245.134.10',
 '178.176.73.137',
 '80.250.77.52',
 '93.92.179.157',
 '95.129.169.201',
 '93.170.247.18',
 '217.76.11.254',
 '217.76.12.174',
 '217.76.13.209',
 '85.115.224.135',
 '85.115.224.148',
 '185.210.141.138',
 '174.250.240.163',
 '162.198.47.79',
 '174.250.241.155',
 '174.250.246.96',
 '62.33.191.144',
 '176.59.44.120',
 '178.125.105.46',
 '73.220.83.204',
 '89.42.205.105',
 '2.132.233.38',


In [10]:
# Получим страну для каждого ip

reuqest_template = 'http://api.sypexgeo.net/json/{}'
country_list = {}
for ip in ip_list:
    json_text = requests.get(reuqest_template.format(ip)).text
    result = json.loads(json_text)
    country_list[ip] = result['country']['name_ru']
country_list

{'46.211.82.121': 'Украина',
 '88.155.87.158': 'Украина',
 '94.153.28.71': 'Украина',
 '94.153.39.188': 'Украина',
 '94.153.51.41': 'Украина',
 '194.44.191.146': 'Украина',
 '46.211.52.51': 'Украина',
 '198.45.117.113': 'Казахстан',
 '37.151.80.79': 'Казахстан',
 '95.82.113.175': 'Казахстан',
 '79.170.137.163': 'Россия',
 '79.170.138.60': 'Россия',
 '80.250.77.52': 'Россия',
 '95.129.169.201': 'Россия',
 '95.129.169.212': 'Россия',
 '90.188.177.197': 'Россия',
 '176.212.96.61': 'Россия',
 '176.59.53.125': 'Россия',
 '67.6.130.230': 'США',
 '176.59.55.106': 'Россия',
 '176.98.70.86': 'Украина',
 '67.44.208.167': 'США',
 '208.98.136.204': 'США',
 '188.234.197.40': 'Россия',
 '74.76.231.192': 'США',
 '178.176.74.170': 'Россия',
 '172.58.29.110': 'США',
 '46.148.132.39': 'Россия',
 '93.157.175.97': 'Россия',
 '109.252.57.7': 'Россия',
 '46.242.15.57': 'Россия',
 '94.245.134.10': 'Россия',
 '178.176.73.137': 'Россия',
 '93.92.179.157': 'Россия',
 '93.170.247.18': 'Россия',
 '217.76.11.254':

In [11]:
# И вернём это в датафрейм

ip_country = pd.DataFrame(list(country_list.items()),
                   columns=['ip', 'country'])
ip_country

,ip,country
0,46.211.82.121,Украина
1,88.155.87.158,Украина
2,94.153.28.71,Украина
3,94.153.39.188,Украина
4,94.153.51.41,Украина
...,...,...
83,107.77.197.172,США
84,174.222.6.202,США
85,76.169.223.121,США
86,46.72.217.127,Россия


In [12]:
# Добавим к общей таблице страну

pays = pays.merge(
    ip_country,
    how='inner',
    on='ip'
)
pays

,event_time_installs,event_date_installs,user_id,ip,device_type,event_time_payments,event_date_payments,revenue_usd,country
0,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,2021-01-05 09:52:02,2021-01-05,10.000000,США
1,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,2021-01-07 03:49:32,2021-01-07,8.000000,США
2,2021-01-01 03:11:29,2021-01-01,216149,96.28.104.236,iPhone X,2021-01-08 07:18:57,2021-01-08,8.000000,США
3,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11,2021-01-01 03:24:11,2021-01-01,9.000000,США
4,2021-01-01 03:13:38,2021-01-01,216151,96.28.48.22,iPhone 11,2021-01-01 03:24:49,2021-01-01,10.000000,США
...,...,...,...,...,...,...,...,...,...
562,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-16 16:23:17,2021-01-16,1.346895,Россия
563,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-16 21:48:23,2021-01-16,1.346895,Россия
564,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-18 14:23:39,2021-01-18,0.660628,Россия
565,2021-01-05 17:13:53,2021-01-05,216848,84.54.44.99,HONOR-HRY-LX1T,2021-01-18 23:16:05,2021-01-18,5.391229,Россия


In [13]:
# Получим суммарный доход для каждой страны

pays_country = pays.groupby(['country']).agg({'revenue_usd':'sum'})
pays_country

,revenue_usd
country,
Армения,25.650000
Беларусь,8.500000
Казахстан,97.895554
Россия,600.947011
США,4520.610000
Украина,567.858678
Филиппины,1.000000


# Проделаем те же манипуляции с ip, только теперь для таблицы с установками

In [14]:
ip_list_installs = installs['ip'].tolist()
ip_list_installs

['96.28.104.236',
 '96.28.48.22',
 '74.56.1.222',
 '70.93.185.219',
 '174.62.104.183',
 '136.152.143.53',
 '107.77.204.66',
 '172.56.23.177',
 '69.57.248.48',
 '94.25.227.184',
 '94.25.179.24',
 '93.170.247.18',
 '93.170.247.18',
 '71.15.206.127',
 '99.61.135.143',
 '223.104.145.68',
 '198.27.189.54',
 '94.245.180.131',
 '176.64.20.103',
 '75.169.146.24',
 '74.132.178.36',
 '173.28.51.231',
 '75.21.229.82',
 '172.58.31.31',
 '97.100.152.88',
 '89.42.205.105',
 '68.10.251.87',
 '85.249.24.81',
 '72.140.212.215',
 '217.76.13.209',
 '72.140.212.215',
 '72.140.212.215',
 '72.140.212.215',
 '79.170.138.60',
 '81.222.185.238',
 '172.125.169.36',
 '89.42.205.105',
 '72.140.212.215',
 '123.19.97.75',
 '188.232.208.186',
 '106.220.108.54',
 '73.220.83.204',
 '104.12.39.222',
 '75.155.125.241',
 '213.230.101.21',
 '8.40.204.210',
 '188.243.17.112',
 '178.172.191.151',
 '109.252.14.20',
 '85.173.132.139',
 '50.110.20.173',
 '94.25.233.139',
 '90.151.42.65',
 '176.59.204.220',
 '73.220.83.204',
 '

In [15]:
reuqest_template = 'http://api.sypexgeo.net/json/{}'
country_list_installs = {}

for ip in ip_list_installs:
    try:
        json_text = requests.get(reuqest_template.format(ip)).text
        result = json.loads(json_text)
        country_list_installs[ip] = result['country']['name_ru']
    except:
        time.sleep(0.1)
    
country_list_installs

{'96.28.104.236': 'США',
 '96.28.48.22': 'США',
 '74.56.1.222': 'Канада',
 '70.93.185.219': 'США',
 '174.62.104.183': 'США',
 '136.152.143.53': 'США',
 '107.77.204.66': 'США',
 '172.56.23.177': 'США',
 '69.57.248.48': '',
 '94.25.227.184': 'Россия',
 '94.25.179.24': 'Россия',
 '93.170.247.18': 'Россия',
 '71.15.206.127': 'США',
 '99.61.135.143': 'США',
 '223.104.145.68': 'Китай',
 '198.27.189.54': 'США',
 '94.245.180.131': 'Россия',
 '176.64.20.103': 'Казахстан',
 '75.169.146.24': 'США',
 '74.132.178.36': 'США',
 '173.28.51.231': 'США',
 '75.21.229.82': 'США',
 '172.58.31.31': 'США',
 '97.100.152.88': 'США',
 '89.42.205.105': 'Казахстан',
 '68.10.251.87': 'США',
 '85.249.24.81': 'Россия',
 '72.140.212.215': 'Канада',
 '217.76.13.209': 'Армения',
 '79.170.138.60': 'Россия',
 '81.222.185.238': 'Россия',
 '172.125.169.36': 'США',
 '123.19.97.75': 'Вьетнам',
 '188.232.208.186': 'Россия',
 '106.220.108.54': 'Индия',
 '73.220.83.204': 'США',
 '104.12.39.222': 'США',
 '75.155.125.241': 'Канад

In [16]:
ip_country_install = pd.DataFrame(list(country_list_installs.items()),
                   columns=['ip', 'country'])
ip_country_install

,ip,country
0,96.28.104.236,США
1,96.28.48.22,США
2,74.56.1.222,Канада
3,70.93.185.219,США
4,174.62.104.183,США
...,...,...
818,84.54.44.99,Россия
819,85.140.22.30,Россия
820,31.47.171.53,Россия
821,176.59.50.238,Россия


In [17]:
# Получим количество пользоватетелей для каждой страны

ip_country_install = ip_country_install.groupby(['country']).agg({'ip':'nunique'})
ip_country_install

,ip
country,
,1
Аргентина,2
Армения,3
Беларусь,15
Боливия,1
Бразилия,1
Великобритания,1
Вьетнам,6
Грузия,2


In [18]:
# Смёржим таблички и сделаем ренейм

result = pays_country.merge(
    ip_country_install,
    how='inner',
    on='country'
).rename(columns={'ip':'users_count'})
result

,revenue_usd,users_count
country,,
Армения,25.650000,3
Беларусь,8.500000,15
Казахстан,97.895554,31
Россия,600.947011,365
США,4520.610000,219
Украина,567.858678,91
Филиппины,1.000000,1


In [19]:
# Получим ARPU

result['ARPU'] = result['revenue_usd'] / result['users_count']
result

,revenue_usd,users_count,ARPU
country,,,
Армения,25.650000,3,8.550000
Беларусь,8.500000,15,0.566667
Казахстан,97.895554,31,3.157921
Россия,600.947011,365,1.646430
США,4520.610000,219,20.642055
Украина,567.858678,91,6.240205
Филиппины,1.000000,1,1.000000


## Проверил один выпадающий ip

In [20]:
installs[installs['ip'] == '5.251.104.10']

,event_time,event_date,user_id,ip,device_type
308,2021-01-02 18:55:51,2021-01-02,216349,5.251.104.10,xiaomi-Redmi Note 5


In [21]:
installs.loc[308:310]

,event_time,event_date,user_id,ip,device_type
308,2021-01-02 18:55:51,2021-01-02,216349,5.251.104.10,xiaomi-Redmi Note 5
309,2021-01-02 18:58:19,2021-01-02,216350,185.160.37.5,iPad Air
310,2021-01-02 19:00:05,2021-01-02,213825,178.125.105.46,Redmi-Redmi Note 9 Pro


In [22]:
json_text = requests.get('http://api.sypexgeo.net/json/185.160.37.5').text
result_test = json.loads(json_text)
result_test


{'ip': '185.160.37.5',
 'city': None,
 'region': None,
 'country': None,
 'error': '',
 'request': -9574,
 'created': '2022.03.31',
 'timestamp': 1648750271}

## Бонусное

In [29]:
def country_group (data):
    if data['ARPU'] <= 1:
        return 'tier-3'
    if data['ARPU'] <= 5:
        return 'tier-2'
    if data['ARPU'] > 5:
        return 'tier-1'

In [34]:
result['top'] = result.apply(country_group, axis=1)
result.sort_values('ARPU', ascending=False)

,revenue_usd,users_count,ARPU,top
country,,,,
США,4520.610000,219,20.642055,tier-1
Армения,25.650000,3,8.550000,tier-1
Украина,567.858678,91,6.240205,tier-1
Казахстан,97.895554,31,3.157921,tier-2
Россия,600.947011,365,1.646430,tier-2
Филиппины,1.000000,1,1.000000,tier-3
Беларусь,8.500000,15,0.566667,tier-3


### Проверка на выпадающий ip - это айпишник, который не выдавал страну (таких много в цикле)